# LIBARY

In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
#import fiona
import pyogrio
import requests
import zipfile
import os
from io import BytesIO
from bs4 import BeautifulSoup

# DOWNLOAD BUILDING DATA

This notebook is to download building data, extract it, then aggregate the data to count the number of building for each census tract.

In [5]:
# URL of the webpage to scrape
url = "https://disasters.geoplatform.gov/USA_Structures/"

def fetch_state_links():
    """Fetches state names and their corresponding links from the webpage."""
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        links = soup.find_all("a", href=True)
        return {link.text.strip(): link["href"] for link in links if "Deliverable" in link["href"]}
    else:
        print("Failed to fetch the webpage. Status code:", response.status_code)
        return {}

def get_link_by_state(state_name, state_links):
    """Returns the link for a given state name."""
    return state_links.get(state_name, "State not found")   

In [11]:
def download_and_extract_zip(state_name, state_links):
    """Downloads and extracts a ZIP file from the given URL."""
    url = get_link_by_state(state_name, state_links)
    parent_dir = os.path.dirname(os.getcwd())
    output_dir = os.path.join(parent_dir, 'Data', 'building_data_gdb')

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        os.makedirs(output_dir, exist_ok=True)
        zip_path = os.path.join(output_dir, f"{state_name}_Structures.zip")
        
        with open(zip_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
        
        os.remove(zip_path)
        print(f"Downloaded, extracted, and deleted ZIP file for {state_name} to {output_dir}")
    else:
        print("Failed to download the ZIP file.")

In [13]:
state_links = fetch_state_links()
# download_and_extract_zip("Alaska", state_links)

In [14]:
for state in state_links:
    download_and_extract_zip(state, state_links)

Downloaded, extracted, and deleted ZIP file for Alabama to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for Alaska to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for American Samoa to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for Arizona to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for Arkansas to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for California to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZIP file for Colorado to c:\Users\ysurya\Documents\EarthquakeDamageModel_Heinz\Data\building_data_gdb
Downloaded, extracted, and deleted ZI

# READ BUILDING DATA

In [2]:
def get_building_data_directory(stateid):
    cwd = os.getcwd()

    # get parent directory
    parent_dir = os.path.dirname(cwd)
    # get the building data directory
    building_data_directory = os.path.join(parent_dir, 'Data', 'building_data_gdb')
    # find all folder in the building data directory
    folders = [f for f in os.listdir(building_data_directory) if os.path.isdir(os.path.join(building_data_directory, f))]
    # get the folder that ends with stateid
    stateid_dir= [f for f in folders if f.endswith(f'{stateid}')][0]

    return os.path.join(building_data_directory, stateid_dir, f'{stateid}_Structures.gdb')

def get_building_data_csv(stateid):
    building_data_directory = get_building_data_directory(stateid)

    # get the csv file
    return os.path.join(building_data_directory, f'{stateid}.csv')

In [3]:
# if false makedir
def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    else:
        print(f"Directory {directory} already exists.")

In [15]:
# read only the specified columns
def read_cols(path):
    cols = ['OCC_CLS', 'PRIM_OCC', 'CENSUSCODE', 'LONGITUDE', 'LATITUDE']
    return gpd.read_file(path, columns=cols)
    
# only read specific columns, it can reduce the memory usage and time for each state    

In [16]:
# Check if a csv file for a state is exists
    # if exists, read it
    # if not, check if the gdb file exists
    # if exists, read it
def read_building_data(stateid):
    building_data_directory = get_building_data_directory(stateid)

    # get the csv file
    csv_path = get_building_data_csv(stateid)

    if os.path.exists(csv_path):
        print(f"Reading {csv_path}")
        return gpd.read_file(csv_path)
    else:
        print(f"{csv_path} does not exist.")
        gdb_path = os.path.join(building_data_directory)
        if os.path.exists(gdb_path):
            print(f"Reading {gdb_path}")
            return read_cols(gdb_path)
        else:
            print(f"{gdb_path} does not exist.")
            print("Please download the gdb file from the USGS website.")

In [17]:
states_data = [
    #("Alabama", "AL"), ("Alaska", "AK"), ("Arizona", "AZ"), ("Arkansas", "AR"),
    ("California", "CA"), ("Colorado", "CO"), ("Connecticut", "CT"), ("Delaware", "DE"),
    ("Florida", "FL"), ("Georgia", "GA"), ("Hawaii", "HI"), ("Idaho", "ID"),
    ("Illinois", "IL"), ("Indiana", "IN"), ("Iowa", "IA"), ("Kansas", "KS"),
    ("Kentucky", "KY"), ("Louisiana", "LA"), ("Maine", "ME"), ("Maryland", "MD"),
    ("Massachusetts", "MA"), ("Michigan", "MI"), ("Minnesota", "MN"), ("Mississippi", "MS"),
    ("Missouri", "MO"), ("Montana", "MT"), ("Nebraska", "NE"), ("Nevada", "NV"),
    ("New Hampshire", "NH"), ("New Jersey", "NJ"), ("New Mexico", "NM"), ("New York", "NY"),
    ("North Carolina", "NC"), ("North Dakota", "ND"), ("Ohio", "OH"), ("Oklahoma", "OK"),
    ("Oregon", "OR"), ("Pennsylvania", "PA"), ("Rhode Island", "RI"), ("South Carolina", "SC"),
    ("South Dakota", "SD"), ("Tennessee", "TN"), ("Texas", "TX"), ("Utah", "UT"),
    ("Vermont", "VT"), ("Virginia", "VA"), ("Washington", "WA"), ("West Virginia", "WV"),
    ("Wisconsin", "WI"), ("Wyoming", "WY")
]

# AGGREGATE BUILDING DATA

In [18]:
# function to remap OCC_CLS and PRIM_OCC
def remap_occupancy_classes(gdf):
    # Define the mapping dictionaries
    building_data = gdf[['BUILD_ID', 'OCC_CLS', 'PRIM_OCC', 'SQFEET', 'FIPS', 'CENSUSCODE', 'LONGITUDE', 'LATITUDE', 
                    'geometry']]
    # mapping the occupancy class
    mapping = {
        'Agriculture':'OTHER', 'Education':'OTHER', 'Residential':'RESIDENTIAL', 'Unclassified':'OTHER',
        'Commercial':'OTHER', 'Government':'OTHER', 'Industrial':'OTHER', 'Utility and Misc':'OTHER',
        'Assembly':'OTHER'
    }
    building_data['OCC_CLS'] = building_data['OCC_CLS'].map(mapping)

    # mapping the primary occupancy
    mapping = {i:'OTHER' for i in building_data['PRIM_OCC'].unique() if i not in ['Single Family Dwelling', 'Multi - Family Dwelling']}
    residential = {'Single Family Dwelling':'SINGLE FAMILY', 'Multi - Family Dwelling':'MULTI FAMILY'}
    mapping.update(residential)
    building_data['PRIM_OCC'] = building_data['PRIM_OCC'].map(mapping)
    return building_data


# function to aggregate the building counts by GEODI, OCC_CLS, PRIM_OCC
def aggregate_building_counts(gdf):
    building_data = remap_occupancy_classes(gdf)
    # group by GEODI, OCC_CLS, PRIM_OCC and sum the counts
    count_building_data = building_data.groupby(['CENSUSCODE', 'OCC_CLS', 'PRIM_OCC']).agg({'BUILD_ID':'count', 'SQFEET':'sum'}).reset_index()
    # rename the columns
    count_building_data = count_building_data.rename(columns={'BUILD_ID':'COUNT'})
    return count_building_data

In [19]:
def pivot_building_data(count_building_data):
    df = count_building_data.copy()

    # Create a pivot table
    df_pivot = df.pivot_table(index="CENSUSCODE", columns=["OCC_CLS", "PRIM_OCC"], values="COUNT", aggfunc="sum", fill_value=0)

    # Flatten MultiIndex columns
    df_pivot.columns = [f"{col[0]}_{col[1]}" for col in df_pivot.columns]
    df_pivot = df_pivot.reset_index()

    return df_pivot

In [ ]:
for state in states_data:
    state_name, stateid = state
    print(f"Reading building data for {state_name}")
    gdf = read_building_data(stateid)
    if gdf is not None:
        count_building_data = aggregate_building_counts(gdf)
        df_pivot = pivot_building_data(count_building_data)
        output_dir = os.path.join(os.path.dirname(os.getcwd()), 'Data', 'building_data_csv')
        create_directory_if_not_exists(output_dir)
        output_path = os.path.join(output_dir, f"{stateid}_building_data.csv")
        df_pivot.to_csv(output_path, index=False)
        print(f"Saved building data for {state_name} to {output_path}")